# LangGraph 101: Building Your First Agent (TypeScript)

Welcome to LangGraph 101! This notebook will walk you through the core concepts of building agents with LangChain and LangGraph using TypeScript.

**What you'll learn:**
- How to interact with language models
- Working with messages and conversation
- Adding tools to extend LLM capabilities
- Building an agent that can reason and act
- Adding memory to maintain context
- Streaming responses for better UX
<br>
<br>
---
<br>

> **Note:** This tutorial uses LangChain v1, which provides the easiest way to start building with LLMs. LangChain agents are built on top of LangGraph, providing durable execution, streaming, human-in-the-loop, and persistence out of the box.


## 🚀 Quick Start Checklist

Before running this notebook, make sure you've completed these steps:

1. ✅ Installed Node.js (v20+) and pnpm
2. ✅ Run `pnpm install` from `/Users/victormoreira/Desktop/demos/langgraph-101-ts`
3. ✅ Created a `.env` file with your `OPENAI_API_KEY`
4. ✅ Installed tslab: `npm install -g tslab && tslab install`
5. ✅ Selected "TypeScript" as your kernel (top-right in VS Code/Jupyter)

**Tips:**
- Wait 2-3 seconds between running cells
- If you get errors, restart the kernel and try again
- Full setup guide is in the [README.md](../../README.md)


## Part 0: Setup & Installation

First, let's install the necessary packages and set up our environment.

> **⚠️ Important Notes:**
> - **Before running this notebook**, make sure you've run `pnpm install` from the project root
> - **Wait a few seconds** between running cells to avoid tslab timing issues
> - If you get a "rebuildTimer" error, **restart the kernel** and try again
> - See the README.md for full troubleshooting guide


In [1]:
// IMPORTANT: Before running this notebook, install dependencies from the project root:
// 
// cd /Users/victormoreira/Desktop/demos/langgraph-101-ts
// pnpm install
// p
// This will install: langchain, @langchain/core, @langchain/langgraph, 
// @langchain/openai, zod, uuid, and dotenv

console.log("✓ Make sure you've run 'pnpm install' from the project root before continuing!");


✓ Make sure you've run 'pnpm install' from the project root before continuing!


In [2]:
// Load environment variables
try {
    await import("dotenv/config");
    console.log("✓ Environment loaded successfully!");
} catch (error) {
    console.log("⚠️  Could not load dotenv. Make sure you've run 'pnpm install' from the project root.");
    console.log("⚠️  You can continue if you've set OPENAI_API_KEY as a system environment variable.");
}

// We'll use OpenAI in this tutorial, but you can swap to any provider!
// Supported providers: OpenAI, Anthropic, Google, and many more
console.log("\n📝 Make sure OPENAI_API_KEY is set in your .env file or environment");


✓ Environment loaded successfully!

📝 Make sure OPENAI_API_KEY is set in your .env file or environment


## Part 1: Your First LLM Call

LangChain provides a **standard model interface** that works across all providers. This means you can easily swap between OpenAI, Anthropic, Google, and other providers without changing your code.

Let's start by initializing a chat model.


In [3]:
import { initChatModel } from "langchain";

// Initialize a chat model - you can easily swap providers!
// Examples: "openai:gpt-4o", "anthropic:claude-3-7-sonnet-latest", "google:gemini-2.0-flash"
const model = await initChatModel("openai:gpt-5-nano");

// Make your first call!
const response = await model.invoke("What is LangChain?");
console.log(response.content);


LangChain is an open-source framework for building applications that use large language models (LLMs). It provides reusable building blocks to connect LLMs to data sources, tools, memory, and external services, so you can create chatbots, question-answering systems, automation agents, and other LLM-powered apps more easily.

Key concepts it offers:
- LLM: Wrappers around language models (e.g., OpenAI, Cohere, local models).
- Prompts and Prompt templates: Structured ways to compose prompts.
- Chains: Sequences of steps (prompt + model calls + post-processing) to transform inputs into outputs.
- Tools: External APIs or functions that an app can call (search, compute, fetch data, etc.).
- Agents: Decision-makers that choose which tools to use and when, based on the LLM’s output.
- Memory: Keeps context across interactions (useful for chatbots and ongoing conversations).
- Vector stores and document loaders: For retrieval-augmented generation (RAG) and knowledge-grounded QA.

Common uses:

### Key Takeaway:
- `initChatModel()` gives you a standardized interface to any LLM provider
- `.invoke()` sends a message and returns a response
- No provider lock-in - swap models easily!


## Part 2: Understanding Messages

**Messages** are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both content and metadata.

There are different message types:
- **SystemMessage** - Instructions for how the model should behave
- **HumanMessage** - User input
- **AIMessage** - Model responses
- **ToolMessage** - Results from tool executions


In [4]:
import { HumanMessage, SystemMessage } from "langchain";

// Create a conversation with different message types
const messages = [
    new SystemMessage("You are a helpful AI assistant that explains technical concepts simply."),
    new HumanMessage("What is an agent?"),
];

const response2 = await model.invoke(messages);
console.log(response2.content);


An agent is any entity that can sense its surroundings and take actions to achieve a goal.

Key ideas:
- Senses: It uses sensors to perceive the environment.
- Acts: It uses actuators or some mechanism to affect the environment.
- Goal-directed: It has objectives or goals it tries to achieve.
- Autonomy: It operates on its own, making decisions based on what it senses and what it knows.

Common types and examples:
- Simple reflex agent: Responds to current inputs with fixed actions (e.g., a light switch that turns on when it’s dark).
- Thermostat: Perceives temperature and turns the heater on/off to reach a target temperature.
- Self-driving car: Senses road conditions, traffic, and pedestrians, and makes driving decisions.
- Software agent (e.g., an email spam filter): Perceives incoming mail and decides whether to mark it as spam.

Components you’ll hear about:
- Sensors: How the agent perceives the world.
- Decision maker: How it chooses actions (rules, models, learning).
- Actuator

### Multi-turn Conversations

Messages make it easy to maintain conversation history:


In [5]:
// Continue the conversation
messages.push(response2);  // Add AI response to history
messages.push(new HumanMessage("Can you give me an example?"));

const response3 = await model.invoke(messages);
console.log(response3.content);


Sure. Here’s a concrete, simple example: a smart home thermostat (the heating system in a room).

- Goal: Keep the room at a comfortable temperature while saving energy.
- Environment: A room with a heater, a temperature sensor, and maybe an occupancy sensor.
- Sensors: Temperature sensor reports current_temp (e.g., 20°C).
- Decision maker: A small set of rules (rule-based). For example, target_temp is 22°C and a margin of 0.5°C.
  - If current_temp < target_temp − 0.5, turn the heater ON.
  - If current_temp > target_temp + 0.5, turn the heater OFF.
  - Otherwise, do nothing.
- Actions (Actuators): The heater (and possibly a fan) are turned ON or OFF, or the heater’s power level is adjusted.
- Knowledge/Model: The desired temperature (target_temp) and the tolerance margin; perhaps a simple policy about when to run the heater.
- Learning (optional): It could learn daily patterns to adjust target_temp for different times of day.

Example run:
- Outside is cold; current_temp is 20°C, tar

### Key Takeaway:
- Messages represent the conversation history
- SystemMessage sets the model's behavior
- Build multi-turn conversations by appending messages to an array


## Part 3: Adding Tools - Extending LLM Capabilities

LLMs are great at language, but they can't access external data or perform actions. **Tools** extend their capabilities. You can give an LLM a list of tools, and when it needs one, it will specify which tool to call. Your job is to execute the tool and feed the results back to the LLM so it can decide what to do next.

You can create a tool just by writing a function with a clear description. LangChain's `tool` function handles formatting the function's information in the LLM's desired format.

Let's create some simple tools:


In [6]:
import { tool } from "langchain";
import { z } from "zod";

// Basic hardcoded tool
const searchMovies = tool(
  async ({ genre }: { genre: string }) => {
    // In a real app, this would query a movie database
    const movies: Record<string, string> = {
      "sci-fi": "Dune, Interstellar, Blade Runner 2049",
      "comedy": "The Grand Budapest Hotel, Superbad, Knives Out",
      "action": "Mad Max: Fury Road, John Wick, Mission Impossible"
    };
    return movies[genre.toLowerCase()] || "No movies found for that genre";
  },
  {
    name: "search_movies",
    description: "Search for movies by genre.",
    schema: z.object({
      genre: z.string().describe("The genre of movies to search for")
    })
  }
);

// More realistic tool that calls an API
const getWeather = tool(
  async ({ latitude, longitude }: { latitude: number; longitude: number }) => {
    const url = "https://api.open-meteo.com/v1/forecast";
    const params = new URLSearchParams({
      latitude: latitude.toString(),
      longitude: longitude.toString(),
      current: "temperature_2m,weather_code",
      temperature_unit: "fahrenheit"
    });

    const response = await fetch(`${url}?${params}`);
    const data = await response.json() as any; // Type assertion for API response
    const weather = data.current;
    const temperature = weather.temperature_2m;
    const weatherCode = weather.weather_code;
    
    return JSON.stringify({
      temperature_fahrenheit: temperature,
      weather_code: weatherCode
    });
  },
  {
    name: "get_weather",
    description: "Get current temperature in Fahrenheit and weather code for given coordinates. Returns JSON with temperature_fahrenheit and weather_code (do not include the code in your response, translate it to plain English)",
    schema: z.object({
      latitude: z.number().describe("Latitude coordinate"),
      longitude: z.number().describe("Longitude coordinate")
    })
  }
);

// Test a tool directly with SF's coordinates
console.log(await getWeather.invoke({ latitude: 37.77, longitude: -122.42 }));


{"temperature_fahrenheit":63.8,"weather_code":0}


### Tool Calling (Function Calling)

Now let's give these tools to the model using `.bindTools()`:

> **Note:** For tool binding, we use `ChatOpenAI` directly instead of `initChatModel()` due to how the configurable model wrapper works in v1.


In [7]:
import { ChatOpenAI } from "@langchain/openai";

// Bind tools to the model
const tools = [getWeather, searchMovies];

// For tool binding, we need to use ChatOpenAI directly (not initChatModel)
const llm = await initChatModel("openai:gpt-4o-mini")

const modelWithTools = llm.bindTools(tools);

const message = "What's the weather like in Seattle?";

// The model can now decide to call tools
const response4 = await modelWithTools.invoke(message);

// Check if the model wants to call a tool
console.log("Tool calls:", response4.tool_calls);


Tool calls: [
  {
    name: 'get_weather',
    args: { latitude: 47.6062, longitude: -122.3321 },
    type: 'tool_call',
    id: 'call_rxHBfz5QLEyp34kdbOd1ndP0'
  }
]


The model returns a **tool call** request with:
- `name`: Which tool to call
- `args`: Arguments to pass to the tool
- `id`: Unique identifier for tracking

Let's execute the tool and continue the conversation:


In [8]:
import { ToolMessage } from "langchain";

// Execute the tool call
if (response4.tool_calls && response4.tool_calls.length > 0) {
    const toolCall = response4.tool_calls[0];
    
    // Call the actual tool
    let result;
    if (toolCall.name === "get_weather") {
        result = await getWeather.invoke(toolCall.args);
    } else if (toolCall.name === "search_movies") {
        result = await searchMovies.invoke(toolCall.args);
    }
    
    // Create a ToolMessage with the result
    const toolMessage = new ToolMessage({
        content: result,
        tool_call_id: toolCall.id
    });
    
    // Continue the conversation with the tool result
    const finalResponse = await modelWithTools.invoke([
        new HumanMessage(message),
        response4,
        toolMessage
    ]);
    
    console.log(finalResponse.content);
}


The weather in Seattle is currently 49.6°F, and it's drizzling light rain.


### Key Takeaway:
- Tools are functions wrapped with the `tool()` function
- Good descriptions help the model know when to use each tool
- Tool calling flow: Model requests tool → Execute tool → Return result → Model synthesizes final response


## Part 4: Building Your First Agent with `createAgent()`

Manually defining a specific sequence of LLM calls and tool calls is tedious and inflexible. Instead, we can use an **agent** that runs this loop:
1. Model decides which tool to call (if any)
2. Tool gets executed
3. Result goes back to model
4. Repeat until task is complete

LangChain makes this easy with `createAgent()` - **build an agent in ~10 lines of code!**
The prebuilt agent handles running the loop described above - you just specify the system prompt and tools.


In [9]:
import { createAgent } from "langchain";

// Create an agent with tools
const agent = createAgent({
    model: "openai:gpt-4o-mini",
    tools: [getWeather, searchMovies],
    systemPrompt: "You are a helpful assistant that can check weather and recommend movies."
});

// Use the agent
const result = await agent.invoke({
    messages: [{ role: "user", content: "What's the weather in NYC? Also recommend some sci-fi movies." }]
});

// Print the conversation
for (const msg of result.messages) {
    console.log(`[${msg.constructor.name}]:`, msg.content);
}


[HumanMessage]: What's the weather in NYC? Also recommend some sci-fi movies.
[AIMessage]: 
[ToolMessage]: {"temperature_fahrenheit":48.3,"weather_code":3}
[ToolMessage]: Dune, Interstellar, Blade Runner 2049
[AIMessage]: The current weather in New York City is 48.3°F, and it's mostly cloudy.

Here are some sci-fi movie recommendations for you:
1. **Dune**
2. **Interstellar**
3. **Blade Runner 2049** 

Enjoy your day!


### What just happened?

The agent automatically:
1. Analyzed the user's request
2. Called `get_weather` for NYC
3. Called `search_movies` for "sci-fi"
4. Synthesized the results into a natural response

You can visualize the agent's structure (note: visualization requires additional setup):


### Key Takeaway:
- `createAgent()` builds a complete agent in ~10 lines
- The agent automatically handles the reasoning → action → observation loop
- Built on LangGraph for production features (persistence, streaming, human-in-the-loop)


## Part 5: Adding Memory & State

Right now, each agent invocation is independent. Let's add **memory** so the agent can maintain context across multiple interactions.

LangGraph uses **checkpointers** to save and restore state:


In [10]:
import { MemorySaver } from "@langchain/langgraph";
import { v4 as uuidv4 } from "uuid";

// Create a checkpointer for memory
const checkpointer = new MemorySaver();

// Create an agent with memory
const agentWithMemory = createAgent({
    model: "openai:gpt-4o-mini",
    tools: [getWeather, searchMovies],
    systemPrompt: "You are a helpful assistant.",
    checkpointer: checkpointer
});

// Create a thread for this conversation
const threadId = uuidv4();
const config = { configurable: { thread_id: threadId } };

// First interaction
const result1 = await agentWithMemory.invoke(
    { messages: [{ role: "user", content: "My name is Alice and I love sci-fi movies." }] },
    config
);

console.log("Response 1:", result1.messages[result1.messages.length - 1].content);

// Second interaction - the agent remembers!
const result2 = await agentWithMemory.invoke(
    { messages: [{ role: "user", content: "What's my name and what movies do I like?" }] },
    config
);
console.log("\nResponse 2:", result2.messages[result2.messages.length - 1].content);


Response 1: Here are some sci-fi movies you might enjoy:

1. **Dune**
2. **Interstellar**
3. **Blade Runner 2049**

Let me know if you need more recommendations!

Response 2: Your name is Alice, and you love sci-fi movies.


### Understanding State & Threads

- **State**: The agent's "memory" - includes message history and any custom data
- **Thread**: A conversation session identified by `thread_id`
- **Checkpointer**: Saves state after each step, enabling memory and error recovery

Each thread is independent:


In [11]:
// New thread - agent won't remember Alice
const newThreadId = uuidv4();
const newConfig = { configurable: { thread_id: newThreadId } };

const result3 = await agentWithMemory.invoke(
    { messages: [{ role: "user", content: "What's my name?" }] },
    newConfig
);
console.log("New thread response:", result3.messages[result3.messages.length - 1].content);


New thread response: I'm sorry, but I don't know your name. Could you please tell me?


### Key Takeaway:
- Checkpointers enable memory across interactions
- Thread IDs separate different conversations
- State persists automatically - no manual state management needed!


## Part 6: Streaming for Better UX

LLMs can take a while to respond. **Streaming** shows progress in real-time, dramatically improving user experience.

LangChain supports multiple streaming modes:


### Streaming Agent Steps


In [12]:
// Stream agent progress with streamMode="updates"
console.log("Streaming agent steps:\n");

for await (const chunk of await agent.stream(
    { messages: [{ role: "user", content: "What's the weather in Boston?" }] },
    { streamMode: "updates" } as any
)) {
    for (const [nodeName, data] of Object.entries(chunk)) {
        console.log(`Step: ${nodeName}`);
        // Type guard to check if data has messages property
        if (data && typeof data === "object" && "messages" in data) {
            const messages = data.messages as any[];
            const message = messages[messages.length - 1];
            if (message.tool_calls && message.tool_calls.length > 0) {
                console.log(`   Tool call: ${message.tool_calls[0].name}`);
            } else if (message.content) {
                const content = message.content.length > 100 
                    ? `${message.content.substring(0, 100)}...` 
                    : message.content;
                console.log(`   Content: ${content}`);
            }
        }
        console.log();
    }
}


Streaming agent steps:

Step: model_request
   Tool call: get_weather

Step: tools
   Content: {"temperature_fahrenheit":46.8,"weather_code":3}

Step: model_request
   Content: The current temperature in Boston is 46.8°F, and the weather is mostly cloudy.



### Streaming LLM Tokens

For a ChatGPT-like experience, stream tokens as they're generated:


In [13]:
// Stream tokens - simplified ChatGPT-like experience
console.log("Streaming tokens:\n");

for await (const [token, metadata] of await agent.stream(
    { messages: [{ role: "user", content: "Tell me about LangGraph in one sentence." }] },
    { streamMode: "messages" } as any
)) {
    // Print any text content we find
    if (token.text) {
        process.stdout.write(token.text);
    }
}

console.log("\n");  // New line at the end


Streaming tokens:

LangGraph is a platform that allows users to create and manage language models using graph-based techniques, facilitating better understanding and manipulation of natural language processing tasks.



### Key Takeaway:
- `streamMode: "updates"` - See each agent step (useful for debugging)
- `streamMode: "messages"` - Stream LLM tokens (ChatGPT-like UX)
- Streaming is built-in - no extra setup required!


## Part 7: Putting It All Together - A Practical Example

Let's build a more realistic agent that combines everything we've learned:


In [14]:
// Create more realistic tools
const getUserPreferences = tool(
  async ({ userId }: { userId: string }) => {
    // Simulate a user database
    const preferences: Record<string, string> = {
      "alice": "Loves sci-fi movies, prefers warm weather destinations",
      "bob": "Enjoys comedy films, likes cold climates for travel"
    };
    return preferences[userId.toLowerCase()] || "No preferences found";
  },
  {
    name: "get_user_preferences",
    description: "Get a user's saved preferences.",
    schema: z.object({
      userId: z.string().describe("The user ID to look up")
    })
  }
);

const bookRecommendation = tool(
  async ({ genre, userPreferences }: { genre: string; userPreferences?: string }) => {
    const recommendations: Record<string, string> = {
      "sci-fi": "Based on your preferences, try: Arrival, Ex Machina, or The Martian",
      "comedy": "Based on your preferences, try: The Big Lebowski, Anchorman, or Bridesmaids"
    };
    return recommendations[genre.toLowerCase()] || "No recommendations available";
  },
  {
    name: "book_recommendation",
    description: "Get personalized movie recommendations based on genre and user preferences.",
    schema: z.object({
      genre: z.string().describe("The movie genre"),
      userPreferences: z.string().optional().describe("Optional user preferences")
    })
  }
);

// Create a helpful assistant agent
const assistant = createAgent({
    model: "openai:gpt-4o-mini",
    tools: [getWeather, getUserPreferences, bookRecommendation],
    systemPrompt: `You are a helpful personal assistant. 
    
    You can:
    - Check weather for any city
    - Look up user preferences
    - Recommend movies based on preferences
    
    Always be friendly and personalize your responses based on user preferences.`,
    checkpointer: new MemorySaver()
});

// Demo conversation
const demoThreadId = uuidv4();
const demoConfig = { configurable: { thread_id: demoThreadId } };

console.log("=".repeat(50));
console.log("PERSONAL ASSISTANT DEMO");
console.log("=".repeat(50) + "\n");

// Interaction 1
console.log("User: Hi, I'm Alice. Can you check my preferences and recommend a movie?\n");
const demoResult1 = await assistant.invoke(
    { messages: [{ role: "user", content: "Hi, I'm Alice. Can you check my preferences and recommend a movie?" }] },
    demoConfig
);
console.log(`Assistant: ${demoResult1.messages[demoResult1.messages.length - 1].content}\n`);

// Interaction 2
console.log("User: Also, what's the weather like in San Francisco?\n");
const demoResult2 = await assistant.invoke(
    { messages: [{ role: "user", content: "Also, what's the weather like in San Francisco?" }] },
    demoConfig
);
console.log(`Assistant: ${demoResult2.messages[demoResult2.messages.length - 1].content}\n`);

console.log("=".repeat(50));


PERSONAL ASSISTANT DEMO

User: Hi, I'm Alice. Can you check my preferences and recommend a movie?

Assistant: Hi Alice! Based on your love for sci-fi movies, I recommend checking out **Arrival**, **Ex Machina**, or **The Martian**. These films are sure to engage you with their intriguing stories and visuals. Enjoy your movie time! If there's anything else you'd like, just let me know.

User: Also, what's the weather like in San Francisco?

Assistant: The weather in San Francisco is currently a pleasant 63.8°F and clear skies. It's a lovely day to explore the city! If you need anything else or have any other questions, feel free to ask!



## Part 8: Next Steps - Exploring LangGraph Primitives

We've been using `createAgent()`, which is built on **LangGraph**. LangGraph gives you full control over agent behavior using three core primitives:

### Core LangGraph Concepts:

1. **State**
   - Shared data structure passed between nodes
   - Represents the agent's "memory"
   - Can include messages, custom data, etc.

2. **Nodes**
   - Functions that process state
   - Each node performs a specific task
   - Examples: call LLM, execute tool, validate input

3. **Edges**
   - Define flow between nodes
   - Can be normal (always go to next node)
   - Or conditional (decide based on logic)


### When to use `createAgent()` vs custom LangGraph?

**Use `createAgent()` when:**
- Building standard ReAct-style agents
- You need quick prototyping
- Default behavior works for your use case

**Use custom LangGraph when:**
- You need custom control flow (e.g., approval workflows)
- Building multi-agent systems
- Implementing human-in-the-loop patterns
- Complex state management requirements

For more advanced patterns, check out:
- [LangGraph Documentation](https://docs.langchain.com/oss/javascript/langgraph/overview)
- [LangChain Academy](https://academy.langchain.com/)
- The `multi_agent.ipynb` notebook in this repo (LangGraph 201)


## 🎉 Congratulations!

You've learned the core concepts of building agents with LangChain and LangGraph:

✅ **Models** - Standardized interface across providers  
✅ **Messages** - Building block of conversations  
✅ **Tools** - Extending LLM capabilities  
✅ **Agents** - Automated reasoning and action loops  
✅ **Memory** - Maintaining context across interactions  
✅ **Streaming** - Real-time user experience  
✅ **LangGraph** - The foundation powering it all

### What's Next?

1. **Build your own agent** with your specific tools and use case
2. **Explore advanced patterns** in the `multi_agent.ipynb` notebook
3. **Add debugging** with [LangSmith](https://smith.langchain.com)
4. **Deploy to production** using LangGraph's persistence and error recovery

### Resources:

- [LangChain Documentation](https://docs.langchain.com/oss/javascript/langchain/overview)
- [LangGraph Documentation](https://docs.langchain.com/oss/javascript/langgraph/overview)
- [LangSmith for Debugging](https://smith.langchain.com)
- [LangChain Academy](https://academy.langchain.com/)
<br>
<br>
---
<br>

**Happy building!**
